In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import mne
from mne.io import read_raw_egi
from mne import concatenate_raws, read_annotations
import numpy as np
import random
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import minmax_scale, scale
from sklearn.model_selection import train_test_split
from aux_func.data_reading import generate_dataset, load_and_slice
from copy import copy

In [2]:
# Zona de declaración de constantes
CHANNELS = 64
WINDOW_WIDTH = 128
WINDOW_STEPS = 16
batch_size = 32

In [3]:
# Cargamos el dataset. Esto devuelve un diccionario que podemos emplear para seleccionar qué tipos de datos queremos usar.
dataset = generate_dataset([[-1],[-1]])

In [4]:
dataset["EEGs_limpieza_CSIC"]["FTD"]["control"][0]

'C:\\Users\\Ceiec01\\OneDrive - UFV\\datasets\\EEGs_Pre_Post_LD\\v1.0.0\\controles\\EEGs_limpieza_CSIC\\CO06_FTD_eeg.npy'

In [5]:
len(dataset["EEGs_limpieza_CSIC"]["FTD"]["control"])

32

In [6]:
pat_reposo = dataset["EEGs_brutos"]["Resting"]["pat_pre"]
controles = dataset["EEGs_brutos"]["Resting"]["control"]

In [7]:
X = pat_reposo + controles
Y = [1]*len(pat_reposo) + [0]*len(controles)

In [8]:
# info = mne.create_info(ch_names=64, sfreq=100)
# raw = mne.io.RawArray(X[200],info)
# raw.plot(title='Data from arrays',
#          show=True, block=True)

In [66]:
import tensorflow as tf

In [67]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [68]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
ds = tf.data.Dataset.from_generator(
                                load_and_slice, 
                                (tf.int64, tf.int64), 
                                (tf.TensorShape([CHANNELS,WINDOW_WIDTH,1]), tf.TensorShape([])), 
                                args = (X,Y,WINDOW_STEPS,WINDOW_WIDTH, [CHANNELS,WINDOW_WIDTH,1]))

ds = ds.batch(batch_size)


ds_train = tf.data.Dataset.from_generator(
                                        load_and_slice, 
                                        (tf.int64, tf.int64), 
                                        (tf.TensorShape([CHANNELS,WINDOW_WIDTH,1]), tf.TensorShape([])), 
                                        args = (X_train,Y_train,WINDOW_STEPS,WINDOW_WIDTH, [CHANNELS,WINDOW_WIDTH,1]))
ds_train = ds_train.batch(batch_size)

ds_test = tf.data.Dataset.from_generator(
                                        load_and_slice,
                                        (tf.int64, tf.int64),
                                        (tf.TensorShape([CHANNELS,WINDOW_WIDTH,1]), tf.TensorShape([])),
                                        args = (X_test,Y_test,WINDOW_STEPS,WINDOW_WIDTH, [CHANNELS,WINDOW_WIDTH,1]))
ds_test = ds_test.batch(batch_size)


In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, TensorBoard

model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(CHANNELS, WINDOW_WIDTH,1)),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [74]:
# Creamos la clase que recibe un Callback, nos permite parar el entrenamiento cuando una medida baje de un valor concreto
class EarlyStoppingByLoss(Callback):
    # Llamada de la inicialización de la función (Verbose está, pero deshabilitado)
    def __init__(self, monitor='loss', value=0.2, verbose=1):
        # Llamamos al padre para sobrecargar métodos
        super(Callback, self).__init__()
        #Establecemos los valores para que sean los parámetros
        self.monitor = monitor
        self.value = value
        self.verbose = verbose
    # Esta función se ejecuta cuando termina una época
    def on_epoch_end(self, epoch, logs={}):
        # Miramos en el log cuanto ha sido el valor que estamos monitorizando
        current = logs.get(self.monitor)
        # Si es menor
        if current <= self.value:
            # que pare el entrenamiento y nos de el feedback
            self.model.stop_training = True
            print("La red ha finalizado el entrenamiento debido al criterio de parada establecido")
            


In [81]:
from tensorflow.keras.callbacks import LambdaCallback
from datetime import datetime
logdir = "logs\\CNN\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1)
modelcallback = ModelCheckpoint('checkpoints/weights.CNN.{epoch:02d}-{val_loss:.2f}.hdf5')
valloss = EarlyStoppingByLoss(monitor='loss', value = 0.02, verbose=1)

# callbacks = callbacks=[LambdaCallback(on_batch_end=lambda batch,logs:print(logs))]
callbacks = [tensorboard_callback,
            modelcallback]

history = model.fit(
    ds_train,
    verbose=0,
    epochs=100,
    validation_data=ds_test,
    callbacks=callbacks,
    steps_per_epoch=len(X)/batch_size,
    validation_steps=len(X)/batch_size,
    shuffle = True #Debido a que los datos están ordenados, si no se hace shuffle no aprende la red
)

In [83]:
metrics = model.evaluate(
    ds_test,
    verbose = 0
)

In [84]:
metrics

[0.0005323748706295035, 1.0]